In [1]:
import pandas as pd
import numpy as np
import itertools as it
import matplotlib.pyplot as plt
from scipy import stats
from collections import defaultdict
import csv

In [2]:
players = pd.read_csv('./data/players.csv')
games = pd.read_csv('./data/games.csv')
plays = pd.read_csv('./data/plays.csv')

In [3]:
yac = defaultdict(dict)
none_set = set()
none_set.add('None')

In [4]:
def getClosest(players, football):
    distances = []
    for index, row in players.iterrows():
        distance = np.linalg.norm(np.subtract([football.x.values[0], football.y.values[0]], [row.x, row.y]))
        distances.append((row.nflId, distance))
    closest = sorted(distances, key= lambda t: t[1])[0][0]
    return closest

In [5]:
def getPlayDetails(gameId, playId):
    play = plays.loc[(plays['gameId'] == gameId) & (plays['playId'] == playId)]
    print(play.playDescription.values[0])

In [6]:
# find receivers x instead of football x
for week_num in range(1, 18):
    print(week_num)
    g_week = games[games['week'] == week_num]
    week_str = './data/week' + str(week_num) + '.csv'
    current_week_data = pd.read_csv(week_str)
    
    for _, g_row in g_week.iterrows():
        GAMEID = g_row.gameId
        cwg_data = current_week_data[current_week_data['gameId'] == GAMEID]
        pg = plays[plays['gameId'] == GAMEID]
        
        for _, p_row in pg.iterrows():
            
            if p_row.passResult == 'C':
                PLAYID = p_row.playId
                curr_play_data = cwg_data[cwg_data['playId'] == PLAYID]
                
                play_events = pd.unique(curr_play_data.event)
#                 if'pass_outcome_caught' not in play_events:
#                     print(GAMEID, PLAYID, play_events)
                
#                 if 'pass_outcome_touchdown' in play_events:
#                     pa_data = curr_play_data[curr_play_data['event'] == 'pass_arrived']
#                     football = pa_data[pa_data['team'] == 'football']
                    
#                     try:
#                         if pa_data.playDirection.values[0] == 'left':
#                             yards = football.x.values[0] - 10
#                         elif pa_data.playDirection.values[0] == 'right':
#                             yards = 110 - football.x.values[0]
#                         else:
#                             print('No play direction?')
#                         if yards > 0: yac[GAMEID][PLAYID] = yards
#                     except:
#                         print(GAMEID, PLAYID, play_events)
                        
                if 'pass_outcome_caught' in play_events:
                    pc_data = curr_play_data[curr_play_data['event'] == 'pass_outcome_caught']
                    football1 = pc_data[pc_data['team'] == 'football']
                    
                    CATCH_FRAME = pc_data.frameId.values[0]
                    after_catch_data = curr_play_data[curr_play_data['frameId'] > CATCH_FRAME]
                    after_catch_events = pd.unique(after_catch_data.event)
                    
                    END_EVENT = ''
                    none_set = set()
                    
                    if 'fumble' in after_catch_events: END_EVENT = 'fumble'
                    elif 'touchdown' in after_catch_events: END_EVENT = 'touchdown'
                    elif 'out_of_bounds' in after_catch_events: END_EVENT = 'out_of_bounds'
                    elif 'tackle' in after_catch_events: END_EVENT = 'tackle'
                    elif 'first_contact' in after_catch_events: END_EVENT = 'first_contact'
                    elif 'pass_lateral' in after_catch_events: END_EVENT = 'pass_lateral'
                    elif len(pd.unique(after_catch_data.event)) == 1 and pd.unique(after_catch_data.event)[0] == 'None':
                        yac[GAMEID][PLAYID] = 0
                        continue
                    else:
                        print(GAMEID, PLAYID, pd.unique(after_catch_data.event))
                        continue
                              
                    end_data = after_catch_data[after_catch_data['event'] == END_EVENT]
                    football2 = end_data[end_data['team'] == 'football']
                    
                    try:
                        if end_data.playDirection.values[0] == 'left':
                            yards = football1.x.values[0] - football2.x.values[0]
                        elif end_data.playDirection.values[0] == 'right':
                            yards = football2.x.values[0] - football1.x.values[0]
                        else:
                            print('No play direction?')
                    except:
                        print(GAMEID, PLAYID)
                        print(football1)
                        print(football2)
                              
                    if yards > 0: 
                        yac[GAMEID][PLAYID] = yards
                    else:
                        yac[GAMEID][PLAYID] = 0
#                     yac[GAMEID][PLAYID] = yards
                        
                        #print(GAMEID, PLAYID, END_EVENT, 'Positive yac on play')
                    #else: 
                        #print(GAMEID, PLAYID, END_EVENT, 'Negative yac on play')
                        
#                 else:
#                     print(GAMEID, PLAYID, 'No pass outcome caught')
                    
                


1
2
3
2018092300 3028 ['pass_outcome_incomplete' 'None']
4
5
6
2018101412 2241
Empty DataFrame
Columns: [time, x, y, s, a, dis, o, dir, event, nflId, displayName, jerseyNumber, position, frameId, team, gameId, playId, playDirection, route]
Index: []
Empty DataFrame
Columns: [time, x, y, s, a, dis, o, dir, event, nflId, displayName, jerseyNumber, position, frameId, team, gameId, playId, playDirection, route]
Index: []
7
8
9
10
11
12
13
14
15
16
17


In [7]:
with open('./intermediate_data/yac_labels.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(['gameId', 'playId', 'yac'])
    for GAME in yac:
        for PLAY, yards in yac[GAME].items():
            writer.writerow([GAME, PLAY, yards])

In [8]:
# for week_num in range(1, 18):
#     print(week_num)
#     g_week = games[games['week'] == week_num]
#     week_str = './data/week' + str(week_num) + '.csv'
#     current_week_data = pd.read_csv(week_str)
    
#     for _, g_row in g_week.iterrows():
#         GAMEID = g_row.gameId
#         cwg_data = current_week_data[current_week_data['gameId'] == GAMEID]
#         pg = plays[plays['gameId'] == GAMEID]
        
#         for PLAY in pg.playId:
#             curr_play_data = cwg_data[cwg_data['playId'] == PLAY]

#             play_events = pd.unique(curr_play_data.event)
#             if 'pass_outcome_caught' in play_events:

#                 catch_data = curr_play_data[curr_play_data['event'] == 'pass_outcome_caught']
#                 football1 = catch_data[catch_data['team'] == 'football']
                
#                 CATCH_FRAME = catch_data.frameId.values[0]
#                 after_catch_data = curr_play_data[curr_play_data['frameId'] >= CATCH_FRAME]
                
#                 after_catch_events = pd.unique(after_catch_data.event)
#                 END_EVENT = ''
#                 if 
#                 if 'tackle' in after_catch_events:
#                     END_EVENT = 'tackle'
#                 elif 'out_of_bounds' in after_catch_events:
#                     END_EVENT = 'out_of_bounds'
#                 elif 'touchdown' in after_catch_events:
#                     END_EVENT = 'touchdown'
#                 elif 'first_contact' in after_catch_events:
#                     END_EVENT = 'first_contact'
#                 else:
#                     pass
# #                     yac[GAMEID][PLAY] = 0
# #                     continue
                    
                    
#                 end_data = after_catch_data[after_catch_data['event'] == END_EVENT]
#                 football2 = end_data[end_data['team'] == 'football']
                
#                 scalar = 1
#                 if catch_data.playDirection.values[0] == 'left':
#                     scalar = -1
#                 elif catch_data.playDirection.values[0] == 'right':
#                     scalar = 1
                
#                 try:
#                     yac[GAMEID][PLAY] = (football2.x.values[0] - football1.x.values[0]) * scalar
#                 except:
#                     print(GAMEID, PLAY, after_catch_events)
#                     getPlayDetails(GAMEID, PLAY)

In [9]:
#                 none_array = ['None']
#                 if len(list(after_catch_events)) == 0 or list(after_catch_events) == none_array:
#                     yac[GAMEID][PLAYID] = 0
#                     continue
#                 elif 'pass_outcome_touchdown' in after_catch_events: END_EVENT = 'pass_outcome_touchdown'
#                 elif 'fumble' in after_catch_events: END_EVENT = 'fumble'
#                 elif 'touchdown' in after_catch_events: END_EVENT = 'touchdown'
#                 elif 'out_of_bounds' in after_catch_events: END_EVENT = 'out_of_bounds'
#                 elif 'tackle' in after_catch_events: END_EVENT = 'tackle'
#                 elif 'first_contact' in after_catch_events: END_EVENT = 'first_contact'
#                 else:
#                     pass
# #                     yac[GAMEID][PLAY] = 0
# #                     continue


#                 end_data = after_catch_data[after_catch_data['event'] == END_EVENT]
#                 football2 = end_data[end_data['team'] == 'football']

#                 scalar = 1
#                 if catch_data.playDirection.values[0] == 'left':
#                     scalar = -1
#                 elif catch_data.playDirection.values[0] == 'right':
#                     scalar = 1

#                 try:
#                     yac[GAMEID][PLAYID] = (football2.x.values[0] - football1.x.values[0]) * scalar
#                 except:
#                     print(GAMEID, PLAYID, after_catch_events)
#                     getPlayDetails(GAMEID, PLAYID)